##Instalations


In [ ]:
!pip install crewai crewai-tools langchain langchain-community langchain-groq faiss-cpu pandas beautifulsoup4 sentence-transformers litellm

  Using cached crewai-1.4.1-py3-none-any.whl.metadata (36 kB)
  Using cached litellm-1.79.3-py3-none-any.whl.metadata (30 kB)
Using cached crewai-1.4.1-py3-none-any.whl (614 kB)
Using cached litellm-1.79.3-py3-none-any.whl (10.4 MB)


##Data Sourcing & Loading

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download canggih/indonesian-food-recipes

In [ ]:
!unzip indonesian-food-recipes

###Data Preprocessing & Chuncking



In [ ]:
import pandas as pd
import glob

RESEP_PATH = "/content/"

resep = glob.glob(RESEP_PATH + "*.csv")
resep_dfs = []

for file in resep:
    try:
        df = pd.read_csv(file, encoding='utf-8')
        resep_dfs.append(df)
    except Exception as e:
        print(f"Gagal load {file}: {e}")

resep_df = pd.concat(resep_dfs, ignore_index=True, sort=False)
resep_df.to_csv("resep.csv", index=False)

In [ ]:
ALL_COLUMNS = [
    'Title', 'Ingredients', 'Steps', 'Loves', 'URL'
]

FILE_NAME = 'resep.csv'

print(f" Loading data from {FILE_NAME} ")

try:
    df = pd.read_csv(FILE_NAME, usecols=ALL_COLUMNS)

    df["Title"] = df["Title"].fillna("Unknown Recipe")
    df["Ingredients"] = df["Ingredients"].fillna("No ingredients listed.")
    df["Steps"] = df["Steps"].fillna("No steps provided.")

    df = df.fillna('N/A')

    print("Data loading complete.")
    print("DataFrame Info:")
    df.info()
    print("\nDataFrame Head:")
    print(df.head())

except Exception as e:
    print(f"An error occurred: {e}")

 Loading data from resep.csv 
Data loading complete.
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15641 entries, 0 to 15640
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        15641 non-null  object
 1   Ingredients  15641 non-null  object
 2   Steps        15641 non-null  object
 3   Loves        15641 non-null  int64 
 4   URL          15641 non-null  object
dtypes: int64(1), object(4)
memory usage: 611.1+ KB

DataFrame Head:
                       Title  \
0         Gurame Saus Padang   
1  Ikan Kembung Bakar Teflon   
2    Mujaer asam pedas manis   
3          Gurame Asam manis   
4       Asam padeh ikan tuna   

                                         Ingredients  \
0  Bahan utama:--1 ekor gurame--Bumbu untuk saus:...   
1  1/2 kg ikan kembung sate, bersihkan--1 buah je...   
2  1/2 kg ikan mujaer (stok gurame habis)--2 buah...   
3  1/2 kg ikan gurame--2 buah wortel (potong sepe.

In [ ]:
from langchain.docstore.document import Document

print("Converting DataFrame to Document objects")

docs = []
for index, row in df.iterrows():
    page_content = f"Nama resep: {row['Title']}\n"
    page_content += f"Bahan: {row['Ingredients']}\n"
    page_content += f"Cara masak: {row['Steps']}"

    metadata = {col: row[col] for col in ALL_COLUMNS}

    docs.append(Document(page_content=page_content, metadata=metadata))

print(f"Successfully created {len(docs)} Document objects.")

Converting DataFrame to Document objects
Successfully created 15641 Document objects.


##Vectorization

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from tqdm.auto import tqdm
from google.colab import userdata

In [ ]:
print("Splitting documents")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(f"Split {len(docs)} documents into {len(splits)} chunks.")

Splitting documents
Split 15641 documents into 19259 chunks.


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-base"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Embedding E5 berhasil di-load!")


/tmp/ipython-input-3759170275.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embedding E5 berhasil di-load!


In [ ]:
import os
import time
from tqdm import tqdm
from langchain_community.vectorstores import FAISS

def vectorize_resep(splits, embeddings, vector_store_path="/content/drive/MyDrive/CollabData/steam_faiss"):
    """
    Vectorizes data resep dari splits menggunakan embeddings,
    disimpan secara persisten di FAISS dengan pemrosesan batch.
    """
    print("\n--- Memulai Vectorization Resep ---")


    faiss_path = os.path.join(vector_store_path, "index.faiss")
    pkl_path = os.path.join(vector_store_path, "index.pkl")

    vector_store = None

    # 2. Cek dan Muat Vector Store Lama (Persisten Logic)
    if os.path.exists(faiss_path) and os.path.exists(pkl_path):
        print(f"Vector store ditemukan di '{vector_store_path}'. Loading...")
        try:
            vector_store = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
            print("Vector store resep loaded berhasil.")
            return vector_store, vector_store.as_retriever(search_kwargs={"k": 5})

        except Exception as e:
            print(f"Error loading Vector Store: {e}")
            print("Melanjutkan dengan membuat Vector Store baru.")
            vector_store = None # Reset untuk memicu pembuatan baru

    print("🆕 Vector store resep tidak ditemukan. Membuat baru...")

    # Pastikan folder ada
    os.makedirs(vector_store_path, exist_ok=True)

    max_attempts = 3
    batch_size = 2000

    for attempt in range(max_attempts):
        try:
            print(f"🔄 Vectorizing dalam batch {batch_size}...")

            for i in tqdm(range(0, len(splits), batch_size)):
                batch = splits[i:i + batch_size]

                if vector_store is None:
                    vector_store = FAISS.from_documents(documents=batch, embedding=embeddings)
                else:
                    vector_store.add_documents(batch, embedding=embeddings)

                print(f"Processed {i + len(batch)}/{len(splits)} chunks.")
                time.sleep(1)

            # 4. Simpan Store Baru
            print(f"Vector store resep berhasil dibuat ({len(splits)} dokumen).")
            print(f"Menyimpan index ke '{vector_store_path}'...")
            vector_store.save_local(vector_store_path)

            return vector_store, vector_store.as_retriever(search_kwargs={"k": 5})

        except Exception as e:
            print(f"Error fatal saat membuat Vector Store resep: {e}")
            return None, None

In [ ]:
# Define the path for the vector store
VECTOR_STORE_PATH = "/content/drive/MyDrive/CollabData/steam_faiss"

# Create the retriever, loading from disk if it exists
vector_store, retriever = vectorize_resep(splits, embeddings, vector_store_path=VECTOR_STORE_PATH)
if not vector_store or not retriever:
    exit("Vector store creation or loading failed. Exiting.")


--- Memulai Vectorization Resep ---
Vector store ditemukan di '/content/drive/MyDrive/CollabData/steam_faiss'. Loading...
Vector store resep loaded berhasil.


##LLM & Tool Setup

In [ ]:
os.environ["SERPER_API_KEY"] = userdata.get('Serper_Api_Key')

print("Secrets loaded.")

Secrets loaded.


In [ ]:
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool

class RecipeSearchTool(BaseTool):
    name: str = "Recipe Database Search Tool"
    description: str = (
        "Searches a private vector database of recipes for titles, ingredients, "
        "and steps. Use this FIRST to find recipes."
    )
    retriever: object

    def _run(self, query: str) -> str:
        print(f"--- Searching Internal Recipe DB for: {query} ---")
        results = self.retriever.invoke(query)

        if not results:
            return "Tidak ditemukan hasil resep untuk pertanyaan tersebut."

        formatted = ["### 🔍 Hasil Pencarian Resep\n"]

        for i, doc in enumerate(results, 1):
            data = doc.metadata
            title = data.get("title", "Tanpa Judul")
            snippet = data.get("snippet", doc.page_content)


            if "--" in snippet:
                parts = [p.strip() for p in snippet.split("--") if p.strip()]
            else:
                parts = [p.strip() for p in snippet.split("\n") if p.strip()]

            bullet_list = "\n".join([f"- {p}" for p in parts])

            formatted.append(
                f"## {i}. {title}\n"
                f"{bullet_list}\n"
            )

        return "\n---\n".join(formatted)

In [ ]:
print("Initializing tools...")

# Instantiate the internal search tool
recipe_search_tool = RecipeSearchTool(retriever=retriever)
print("RecipeSearchTool initialized.")

# Instantiate the web search tool
serper_tool = SerperDevTool()
print("SerperDevTool initialized.")

Initializing tools...
RecipeSearchTool initialized.
SerperDevTool initialized.


In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata
from crewai import  LLM
import os

os.environ["OPENAI_API_KEY"] = "dummy-key"
os.environ["OPENAI_MODEL_NAME"] = "dummy-model"


os.environ["GROQ_API_KEY"] = userdata.get("Groq_API_Key")

llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0.7
)


print("ChatGroq LLM initialized.")

ChatGroq LLM initialized.


##Agent Definiton

In [ ]:
from crewai import Agent

print("Defining Recipe Agent")

recipe_expert = Agent(
    role='Ahli Resep & Kuliner Indonesia',
    goal=(
        "Memberikan jawaban lengkap dan akurat mengenai resep makanan Indonesia. "
        "Pertama, gunakan 'Recipe Search Tool' untuk mencari informasi dari database resep. "
        "Kemudian rangkum hasil pencarian dan berikan penjelasan yang mudah dipahami "
        "Memberikan jawaban lengkap, akurat, dan terstruktur tentang resep makanan Indonesia. "
        "Mulai dari daftar bahan, langkah memasak yang jelas, terurut dan mudah diikuti,  tips memasak, dan variasi rasa. "
        "Jika data tool tidak memiliki langkah memasak, lengkapi langkah berdasarkan pengetahuan kuliner umum."
    ),
    backstory=(
        "Kamu adalah seorang chef profesional yang ahli dalam masakan tradisional dan modern Indonesia. "
        "Kamu memiliki akses ke sebuah database resep lengkap berbahasa Indonesia yang berisi bahan, langkah, "
        "serta penjelasan masakan. Kamu juga mampu menjelaskan langkah dan tips memasak dengan cara yang ringkas dan jelas."
        "Kamu juga memberikan rekomendasi masakan berdasarkan bahan yang tersedia."
        "Tugasmu adalah membantu pengguna menemukan resep terbaik berdasarkan kebutuhan mereka."
    ),
    tools=[recipe_search_tool, serper_tool],
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

print("Agent 'recipe_expert' berhasil dibuat.")


Defining Recipe Agent
Agent 'recipe_expert' berhasil dibuat.


In [ ]:
from crewai import Task

print("Defining Task for Food Chatbot")

recipe_query_task = Task(
    description=(
        "Pengguna bertanya: {pertanyaan}."
        "Tugas kamu adalah memberikan jawaban lengkap dengan mengikuti proses tiga langkah berikut:\n"
        "1. **Pencarian Internal (WAJIB):** Gunakan 'Alat Pencarian Resep' untuk menemukan "
        "resep yang relevan berdasarkan judul, bahan, atau jenis masakan.\n"
        "2. **Pencarian Web (OPSIONAL):** Jika dibutuhkan, gunakan alat 'Search the internet' "
        "untuk mencari informasi tambahan seperti tips memasak, variasi resep, atau teknik "
        "yang sedang tren.\n"
        "3. **Sintesis:** Gabungkan seluruh informasi dari database internal dan web.\n"
        "Berikan daftar rekomendasi resep, jelaskan alasan kecocokannya (berdasarkan database), "
        "dan tambahkan informasi tambahan dari web jika relevan."
    ),

    expected_output=(
        "Jawaban yang lengkap, ramah, terstruktur rapi, dan mudah dipahami. "
        "Jawaban harus berisi:\n"
        "- Daftar rekomendasi resep.\n"
        "- Penjelasan mengapa resep tersebut cocok dengan pertanyaan pengguna.\n"
        "- Informasi tambahan (opsional) dari web seperti tips memasak atau variasi resep.\n"
        "- Jika pengguna mencari sesuatu yang tidak spesifik, berikan beberapa inspirasi masakan."
    ),

    agent=recipe_expert
)

print("Food chatbot task defined.")


Defining Task for Food Chatbot
Food chatbot task defined.


In [ ]:
from crewai import Crew, Process

food_crew = Crew(
    agents=[recipe_expert],
    tasks=[recipe_query_task],
    process=Process.sequential,
    llm=llm,
    verbose=True,
)

print("Crew assembled.")


Crew assembled.


In [ ]:
def chatbot(user_question):
    print(f"--- Kicking off Food Recipe Crew for: {user_question} ---")

    response = food_crew.kickoff(inputs={'pertanyaan': user_question})
    return response


##Execution

In [ ]:
print("\nChatbot Resep Siap! Ketik pertanyaan Anda.\n")

while True:
    user_question = input("Anda: ")

    if user_question.lower() in ["exit", "quit", "keluar"]:
        print("Bot: Terima kasih! Sampai jumpa.")
        break

    response = chatbot(user_question)
    print("\nBot:", response)
    print("\n---------------------------------------\n")


Chatbot Resep Siap! Ketik pertanyaan Anda.

Anda: resep ayam geprek
--- Kicking off Food Recipe Crew for: resep ayam geprek ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 13e774b5-0880-422a-92bd-e6cf835d1d25                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Ahli Resep & Kuliner Indonesia                                                                          │
│                                                                                                                 │
│  Task: Pengguna bertanya: resep ayam geprek.Tugas Anda adalah memberikan jawaban lengkap dengan mengikuti       │
│  proses tiga langkah berikut:                                                                                   │
│  1. **Pencarian Internal (WAJIB):** Gunakan 'Alat Pencarian Resep' untuk menemukan resep yang relevan           │
│  berdasarkan judul, bahan, atau jenis masakan.                                                                  │
│  2. **Pencarian Web (OPSIONAL):** Jika dibutuhkan, gunakan alat 'Search the internet' untuk mencari informasi   │
│  tambahan seperti tips memasak, variasi resep, atau teknik yang sedang tren.                                    │
│  3. **Sintesis:** Gabungkan seluruh informasi dari database internal dan web.                                   │
│  Berikan daftar rekomendasi resep, jelaskan alasan kecocokannya (berdasarkan database), dan tambahkan           │
│  informasi tambahan dari web jika relevan.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Ahli Resep & Kuliner Indonesia                                                                          │
│                                                                                                                 │
│  Thought: Dan ayam geprek siap di sajikan--                                                                     │
│  ---                                                                                                            │
│  Thought: Saya telah menemukan beberapa resep ayam geprek yang relevan di database internal, sekarang saya      │
│  dapat memilih beberapa resep yang paling cocok dengan pertanyaan pengguna dan memberikan penjelasan mengapa    │
│  resep tersebut cocok.                                                                                          │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Ahli Resep & Kuliner Indonesia                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Berikut beberapa rekomendasi resep ayam geprek yang cocok dengan pertanyaan pengguna:                          │
│  1. Ayam Geprek dengan bahan: 3 potong ayam, 7 buah cabai merah, 1 buah bawang putih, 15 sdm tepung terigu, 1   │
│  sdt backing powder, 15 sdm tepung bumbu, Air es, 1 butir telur ayam, 1 sdm garam, 1 sdt gula pasir.            │
│  2. Ayam Geprek dengan bahan: 4 potong ayam, 8 siung bawang putih dihaluskan, 4 sdm tepung terigu, 1 sdm        │
│  tepung maizena, Secukupnya garam, Secukupnya merica, Secukupnya kaldu ayam bubuk, 1 butir telur, Bahan         │
│  geprek: 4 siung bawang putih, 10 buah cabe rawit, 1 sdt garam, 1 sdt gula.                                     │
│  3. Ayam geprek dengan bahan: 1 kg ayam, 5 buah bawang putih, 1 buah jeruk nipis, Bahan sambal: 10 buah cabe    │
│  rawit setan, 3 buah bawang putih, 1 bungkus tepung serbaguna, Air es, Minyak goreng, Garam, Gula.              │
│  4. Ayam Geprek dengan bahan: 1/4 kg sayap ayam, 2 siung bawang putih, 1 bungkus tepung bumbu serbaguna, 2 sdm  │
│  tepung beras, Secukupnya garam, Air, Bumbu sambal: 10 buah cabe kecil, 1 siung bawang putih, Sedikit garam,    │
│  Minyak untuk menggoreng.                                                                                       │
│  5. Ayam Geprek dengan bahan: 4 potong Ayam, 3 sdm tepung bumbu ayam, 1 sdt garam, 6 biji bawang merah, 3 biji  │
│  bawang putih, secukupnya Air, Lombok prawit 12 biji (tergantung selera).                                       │
│  Resep-resep di atas cocok dengan pertanyaan pengguna karena menggunakan bahan-bahan yang umum digunakan dalam  │
│  masakan Indonesia dan memiliki cara memasak yang relatif sederhana. Selain itu, resep-resep di atas juga       │
│  memiliki variasi rasa yang berbeda-beda, sehingga pengguna dapat memilih resep yang paling sesuai dengan       │
│  selera mereka.                                                                                                 │
│  Beberapa tips memasak ayam geprek yang dapat membantu pengguna adalah:                                         │
│  * Pilih daging ayam yang segar dan berkualitas.                                                                │
│  * Gunakan tepung yang tepat untuk membuat ayam geprek yang crispy.                                             │
│  * Marinasi ayam sebelum digoreng untuk membuat rasa lebih kuat.                                                │
│  * Goreng ayam dengan minyak yang cukup untuk membuat ayam geprek yang crispy.                                  │
│  * Tambahkan sambal atau saus sesuai selera untuk membuat rasa lebih pedas atau manis.                          │
│  Dengan mengikuti resep-resep di atas dan tips memasak ayam geprek, pengguna dapat membuat ayam geprek yang     │
│  lezat dan memuaskan.                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 331d7d5d-d411-46cb-927c-c3a28b199033                                                                     │
│  Agent: Ahli Resep & Kuliner Indonesia                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Anda: exit
Bot: Terima kasih! Sampai jumpa.
